## Attention

複数のデータの中から重要なデータに注目する仕組み。  
attentionを用いることで、decoderが、encoderが出力した情報の中の重要な情報に注目するようになる。

前章で作成した翻訳モデルは、入力文をencoderによって固定長のベクトルに変換し、それをdecoderに渡すことで、入力文に基づいた出力文を生成した。  
encoderがRNNであるとき、encoderは各時間で隠れ状態を出力する。この中から最後の時間の隠れ状態のみをdecoderに渡していたものがこれまでのseq2seqである。

このとき、encoderが出力する全ての隠れ状態を利用したいと考える。その方が入力文の多くの情報を参照でき、より適切な出力が得られそうだ。attentionはそれを実現する。

In [1]:
from typing import List
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 学習データ

In [3]:
textfile_ja = 'data/kyoto_ja_10000.txt'
textfile_en = 'data/kyoto_en_10000.txt'
tokenizer_prefix_ja = 'models/tokenizer_kyoto_ja_10000'
tokenizer_prefix_en = 'models/tokenizer_kyoto_en_10000'

In [4]:
with open(textfile_en) as f:
    data_en = f.readlines()

with open(textfile_ja) as f:
    data_ja = f.readlines()

n_data = len(data_en)
print('num of data:', n_data)

sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')
unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()
n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of data: 10000
num of vocabrary (ja): 8000
num of vocabrary (en): 8000


In [5]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

In [6]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja
        x_dec = en[:-1]
        y_dec = en[1:]
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch):
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)


---

## Attention

attention機構について詳しく見ていく。

といっても、そんなに難しいことはない。  
複数のデータがあった時に、各データに重要度を割り当てるだけである。これは重みと呼ばれ、$w_i$で表すことにする。  
重みの総和は1になるように正規化する。正規化前の値はスコアと呼んだりする。

重要度は別の何らかのデータに基づいて設定される。一般的に、そのデータとの内積を取ることが多い。

3つの隠れ状態$h_i$があるとする。

In [7]:
batch_size, hidden_size = 3, 5
hs = torch.randn(batch_size, hidden_size)
hs

tensor([[-1.6353, -0.5284,  0.2080,  0.8854,  1.8761],
        [-0.2456,  2.2601,  2.0749,  1.7099,  2.3480],
        [ 1.1584, -0.7575, -2.3605, -1.0838, -1.2678]])

隠れ状態$h_i$と同じサイズの適当なデータ$x$があるとする。

In [8]:
x = torch.randn(5)
x

tensor([ 0.3501,  0.3938,  0.0569,  0.8265, -1.1349])

$x$と、全ての隠れ状態$h_i$で内積を取り、softmaxで正規化する。

In [9]:
scores = torch.tensor([h @ x for h in hs])
weights = F.softmax(scores, dim=-1)
weights

tensor([0.0457, 0.2867, 0.6677])

スコアはこれでもOK

In [10]:
scores = hs @ x

この3つが重みで、$x$にとっての$h_i$の重要度を表す。

この重みで$h_i$の重み付き和をとることで、$x$にとっての重要度を反映したただ1つの隠れ状態を得ることが出来る。

In [11]:
# 重みをかける
weighted_hs = torch.stack([w * h for w, h in zip(weights, hs)])
weighted_hs

tensor([[-0.0747, -0.0241,  0.0095,  0.0404,  0.0857],
        [-0.0704,  0.6479,  0.5948,  0.4902,  0.6731],
        [ 0.7734, -0.5058, -1.5760, -0.7236, -0.8465]])

In [12]:
# 和をとる
h = weighted_hs.sum(dim=0)
h

tensor([ 0.6283,  0.1180, -0.9717, -0.1930, -0.0877])

以下のようにまとめられる。

In [13]:
h = weights @ hs # 重み付き和
h

tensor([ 0.6283,  0.1180, -0.9717, -0.1930, -0.0877])

seq2seqのdecoderでは、このattentionを利用して、encoderが出力した全ての隠れ状態を参照する。  

encoderの隠れ状態の数はencoderへの入力の数によって変わる。ただ、可変長のデータをモデル内で扱うことは難しい。  
そこで、attentionを用いて1つの固定長のベクトルに変換する。decoderでの演算時に、decoderのRNNから出力された隠れ状態を用いてencoderの隠れ状態の重要度を計算し、重み付き和をとる。こうすることで、encoderの隠れ状態から注目すべき重要な情報を都合よく抽出した固定長のベクトルを得ることが出来る。後はそれを以降の層に渡すだけ。

なお、重みが正しく着目すべき点を表すかは、学習させてみないと分からない。  
この目的も、学習前の段階では期待に過ぎない。この仕組みを取り入れて学習させれば、次第に適切な重みが出力されるようになり、適切な出力が得られるようになるだろう。そうだといいな、ってだけ。

また、重みを求める関数が内積でないといけない理由はない。2つのベクトルからスカラーを得る関数であれば何でもよい。  
内積は類似度を測ることができ、類似度が高いものに着目するという意味では適切に見えるが、そもそもベクトル空間が異なるので、それらの類似度は意味を持たない。  
重みを求める関数を内積として学習を進めれば、重要度が高くなるべきタイミングでその2つのベクトルが類似するように学習される、というだけ。

ただ実際はほとんどの場合で内積が使われる。それは内積という計算がシンプルだからってだけ。

### Attention層

decoderの中の、attentionによって都合のいい隠れ状態を出力する部分は1つの層として見られる。  
実装してみよう。

In [14]:
class Attention(nn.Module):
    def forward(self, x, hs):
        """
        x: (batch_size, seq_len_dec, hidden_size)
        hs: (batch_size, seq_len_enc, hidden_size)
        """
        scores = x @ hs.mT # (batch_size, seq_len_dec, seq_len_enc)
        weights = F.softmax(scores, dim=-1)
        h = weights @ hs # (batch_size, seq_len_dec, hidden_size)
        return h

In [15]:
batch_size, seq_len_dec, seq_len_enc, hidden_size = 2, 3, 4, 5
x = torch.randn(batch_size, seq_len_dec, hidden_size)
hs = torch.randn(batch_size, seq_len_enc, hidden_size)

attention = Attention()
h = attention(x, hs)
h.shape

torch.Size([2, 3, 5])


---

## MASK

padトークンがattentionの計算に含まれてしまうことを回避する。  
maskをかけてpadトークンに対応する重みが0になるようにする。

スコアに対して、対応する位置の値を$-\infty$にする。そうすればsoftmaxを計算したときにその部分が0になる。

こんなスコアがあったとする。

In [16]:
scores = torch.randn(5)
scores

tensor([ 0.3461, -0.4953, -0.3809, -2.0183, -0.8126])

後ろの2つがpadトークンだったとすると、こんな感じでmaskをかけてやればいい。

In [17]:
mask = [False, False, False, True, True]
scores[mask] = -torch.inf
scores

tensor([ 0.3461, -0.4953, -0.3809,    -inf,    -inf])

こう書いてもいい。

In [18]:
scores = torch.randn(5)
mask = torch.tensor([0, 0, 0, 1, 1])
scores.masked_fill_(mask, -torch.inf)
scores

tensor([-0.3672,  0.1310,  0.3589,    -inf,    -inf])

後はこれをsoftmaxに通す。

In [19]:
weights = F.softmax(scores, dim=-1)
weights

tensor([0.2122, 0.3492, 0.4386, 0.0000, 0.0000])

できた。これでpadトークンが無視されるようになる。

層としても実装する。

In [20]:
class Attention(nn.Module):
    def forward(self, x, hs, mask=None):
        """
        x: (batch_size, seq_len_dec, hidden_size)
        hs: (batch_size, seq_len_enc, hidden_size)
        mask: (batch_size, seq_len_enc), bool, padトークンの位置
        """
        scores = x @ hs.mT # (batch_size, seq_len_dec, seq_len_enc)
        if mask is not None:
            scores.masked_fill_(mask.unsqueeze(1), -torch.inf) # maskを適用
        weights = F.softmax(scores, dim=-1)
        h = weights @ hs # (batch_size, seq_len_dec, hidden_size)
        return h


---

## Attentionを用いたSeq2Seq

seq2seqにattention層を取り入れて翻訳モデルを作ってみよう。

### Encoder

In [21]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_positions = x == eos_id # (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, h = self.rnn(x) # hs: (batch_size, seq_len, hidden_size)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
        hs = self.fc(hs) # (batch_size, seq_len, hidden_size)
        h = hs[eos_positions].unsqueeze(0) # (1, batch_size, hidden_size)
        return hs, h

### Decoder

前章のものにattention層を追加する。attention層の前後は残差結合で繋ぐ。

In [27]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.attention = Attention()
        self.fc = nn.Linear(hidden_size * 2, n_vocab)

    def forward(self, x, h_enc, hs_enc, mask=None):
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        hs, h = self.rnn(x, h_enc)
            # hs: (batch_size, seq_len, hidden_size)
            # h: (1, batch_size, hidden_size)
        h = self.attention(hs, hs_enc, mask) # (batch_size, seq_len, hidden_size)
        z = torch.cat([hs, h], dim=-1) # (batch_size, seq_len, hidden_size * 2)
        y = self.fc(z) # (batch_size, seq_len, n_vocab)
        return y, h

### Seq2Seq

全ての隠れ状態とpadトークンの位置をdecoderに渡すようにする。

In [28]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        mask = x_enc == pad_id
        hs, h = self.encoder(x_enc)
        y, _ = self.decoder(x_dec, h, hs, mask)
        return y

### 学習

In [32]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

def eval_model(model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x_enc, x_dec, y_dec in test_loader:
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)
            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            total_loss += loss.item()
    loss = total_loss / len(test_loader)
    return loss

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_loss = eval_model(model)
            prog.memo(f'test: {test_loss:.5f}', no_step=True)
        prog.memo()

In [35]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [36]:
train(model, optimizer, n_epochs=50, prog_unit=5)

  1-5/50: ############################## 100% [00:03:17.08] loss train: 5.14457, test: 4.57513 
 6-10/50: ############################## 100% [00:03:11.39] loss train: 3.91416, test: 4.16876 
11-15/50: ############################## 100% [00:03:09.16] loss train: 3.32283, test: 3.98527 
16-20/50: ############################## 100% [00:03:10.80] loss train: 2.88578, test: 3.91809 
21-25/50: ############################## 100% [00:03:09.96] loss train: 2.53864, test: 3.90768 
26-30/50: ############################## 100% [00:03:11.72] loss train: 2.24531, test: 3.94121 
31-35/50: ############################## 100% [00:03:13.58] loss train: 1.99422, test: 4.00357 
36-40/50: ############################## 100% [00:03:12.77] loss train: 1.76944, test: 4.08644 
41-45/50: ############################## 100% [00:03:31.26] loss train: 1.57332, test: 4.18753 
46-50/50: ############################## 100% [00:03:11.87] loss train: 1.39651, test: 4.30312 


### 翻訳

In [37]:
unk_id = sp_en.unk_id() # UNKのID
def token_sampling(y: List[float]) -> int:
    """モデルの出力から単語をサンプリングする"""
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    hs, h = model.encoder(in_ids)
    next_token = bos_id

    token_ids = []
    for _ in range(max_len):
        x = torch.tensor([[next_token]], device=device)
        y, h = model.decoder(x, h, hs)
        y = y[0]
        if decisive:
            next_token = y.argmax().item()
        else:
            next_token = token_sampling(y)
        token_ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = sp_en.decode(token_ids)
    return sentence

In [38]:
n = 5
for x, t in zip(data_ja[:n], data_en[:n]):
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: 駅情報

output: North Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo Station, Jo
answer: Information


input: 三条京阪駅（さんじょうけいはんえき）は、京都市東山区にある、京都市営地下鉄東西線の鉄道駅。

output: Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama Station Higashiyama St

In [39]:
sentences = [
    'この駅は京都市内の中心部にあります。',
    '京都'
]

In [40]:
for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: この駅は京都市内の中心部にあります。
output: A---------------------------------------------------------------------------------------------------

input: 京都
output: Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s Kyoto Station (-s

